In [1]:
import torch
import torch.nn as nn

In [3]:
class Model(nn.Module):

  def __init__(self, num_features):
    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    out = self.linear(features)
    out = self.sigmoid(out)
    return out

In [4]:
# Create dataset
features = torch.rand(10,5)

# Create model
model = Model(features.shape[1])

# Call model forward pass
# don't do like model.forward(features) instead do the following
model(features)



tensor([[0.5626],
        [0.4971],
        [0.5716],
        [0.5669],
        [0.6270],
        [0.4887],
        [0.4935],
        [0.5406],
        [0.5399],
        [0.5051]], grad_fn=<SigmoidBackward0>)

In [5]:
# Show model weights

model.linear.weight

Parameter containing:
tensor([[ 0.2643, -0.1814,  0.0483, -0.1252, -0.3618]], requires_grad=True)

In [6]:
# Show model bias

model.linear.bias

Parameter containing:
tensor([0.3948], requires_grad=True)

In [7]:
# To vizualise the model summary use torchinfo library

!pip install torchinfo

In [8]:
from torchinfo import summary

summary(model, input_size=(10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 1]                   6
├─Sigmoid: 1-2                           [10, 1]                   --
Total params: 6
Trainable params: 6
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

# Neural Network with a hidden layer

In [9]:
class HiddenLayerModel(nn.Module):

  def __init__(self, num_features):
    super().__init__()
    self.linear1 = nn.Linear(num_features, 3)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(3, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    out = self.linear1(features)
    out = self.relu(out)
    out = self.linear2(out)
    out = self.sigmoid(out)

    return out

In [10]:
features = torch.rand(10,5)

model = HiddenLayerModel(num_features=features.shape[1])

model(features)

tensor([[0.4776],
        [0.4776],
        [0.4776],
        [0.4776],
        [0.4846],
        [0.4776],
        [0.4796],
        [0.4776],
        [0.4776],
        [0.4807]], grad_fn=<SigmoidBackward0>)

In [11]:
summary(model, input_size=(10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
HiddenLayerModel                         [10, 1]                   --
├─Linear: 1-1                            [10, 3]                   18
├─ReLU: 1-2                              [10, 3]                   --
├─Linear: 1-3                            [10, 1]                   4
├─Sigmoid: 1-4                           [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

# Sequential Container

In [12]:
class SequentialModel(nn.Module):

  def __init__(self, num_features):
    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(num_features, 3),
        nn.ReLU(),
        nn.Linear(3, 1),
        nn.Sigmoid()
    )

  def forward(self, features):
    out = self.network(features)
    return out


In [13]:
features = torch.rand(10,5)

model = SequentialModel(num_features=features.shape[1])

model(features)

tensor([[0.3654],
        [0.3654],
        [0.3636],
        [0.3549],
        [0.3654],
        [0.3601],
        [0.3654],
        [0.3639],
        [0.3643],
        [0.3654]], grad_fn=<SigmoidBackward0>)

In [14]:
summary(model, input_size=(10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
SequentialModel                          [10, 1]                   --
├─Sequential: 1-1                        [10, 1]                   --
│    └─Linear: 2-1                       [10, 3]                   18
│    └─ReLU: 2-2                         [10, 3]                   --
│    └─Linear: 2-3                       [10, 1]                   4
│    └─Sigmoid: 2-4                      [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

# Training Pipeline - Breast Cancer Dataset

In [15]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [16]:
df = pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [17]:
# Drop useless columns like id and Unnamed:32

df.drop(columns=["id", "Unnamed: 32"], inplace=True)
df

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [18]:
X = df.iloc[:, 1:].to_numpy()
y = df.iloc[:, 0].to_numpy()

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Scaling the columns
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
# Encoding the target
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [36]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [55]:
class SimpleNN(nn.Module):

  def __init__(self, num_features):
    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    output = self.linear(features)
    output = self.sigmoid(output)
    return output


In [56]:
learning_rate = 0.1
num_epochs = 80
loss_function = nn.BCELoss()

In [57]:
model = SimpleNN(num_features=X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):

  # forward pass
  y_pred = model(X_train_tensor)

  # loss calc
  loss = loss_function(y_pred, y_train_tensor.view(-1, 1))

  # clear gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # parameters update
  optimizer.step()


  print(f"Epoch: {epoch+1} Loss: {loss.item()}")

Epoch: 1 Loss: 0.6906918883323669
Epoch: 2 Loss: 0.5269992351531982
Epoch: 3 Loss: 0.44111400842666626
Epoch: 4 Loss: 0.38836199045181274
Epoch: 5 Loss: 0.3519763946533203
Epoch: 6 Loss: 0.32494834065437317
Epoch: 7 Loss: 0.3038373589515686
Epoch: 8 Loss: 0.28674256801605225
Epoch: 9 Loss: 0.272520512342453
Epoch: 10 Loss: 0.2604379653930664
Epoch: 11 Loss: 0.25000062584877014
Epoch: 12 Loss: 0.2408619523048401
Epoch: 13 Loss: 0.2327706664800644
Epoch: 14 Loss: 0.2255396544933319
Epoch: 15 Loss: 0.21902616322040558
Epoch: 16 Loss: 0.213119238615036
Epoch: 17 Loss: 0.20773087441921234
Epoch: 18 Loss: 0.20279037952423096
Epoch: 19 Loss: 0.19824010133743286
Epoch: 20 Loss: 0.19403238594532013
Epoch: 21 Loss: 0.19012753665447235
Epoch: 22 Loss: 0.18649202585220337
Epoch: 23 Loss: 0.18309733271598816
Epoch: 24 Loss: 0.17991903424263
Epoch: 25 Loss: 0.17693601548671722
Epoch: 26 Loss: 0.174129918217659
Epoch: 27 Loss: 0.17148469388484955
Epoch: 28 Loss: 0.16898615658283234
Epoch: 29 Loss: 0.

In [59]:
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.6).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f"Accuracy: {accuracy}")

Accuracy: 0.5366266369819641
